In [31]:
import math
import numpy as np

### Generating RK4 Cairo program

In [32]:
class RK4CairoBuilder:
    
    def __init__(self, dim=2):
        self.dim = dim
        self.TAB = "    "
        
    def generate(self, print_on_screen=True):
        lines = []
        lines += self._gen_eval()
        lines += self._gen_rk4()
        lines += self._gen_dynamics_util()
        lines += self._gen_fp_util()
        
        print("===============================================")
        print("======== GENERATED CAIRO PROGRAM BELOW ========")
        print("===============================================")
        for line in lines:
            print(line)
    
    def _gen_rk4(self):
        lines = []
        
        lines.append("# Generated Runge-Kutta 4th-order method for Dynamics state")
        lines.append("func rk4 {range_check_ptr} (")
        lines.append("        t : felt,")
        lines.append("        dt : felt,")
        lines.append("        state : Dynamics")
        lines.append("    ) -> (")
        lines.append("        state_nxt : Dynamics")
        lines.append("    ):")
        lines.append("    alloc_locals")
        lines.append("    # k1 stage")
        lines.append("    local k1_state : Dynamics            = state")
        lines.append("    let (local k1_state_diff : Dynamics) = eval (k1_state)")
        lines.append("    let (local k1 : Dynamics)            = dynamics_mul_fp (k1_state_diff, dt)")
        lines.append("")
        lines.append("    # k2 stage")
        lines.append("    let (local k1_half : Dynamics)       = dynamics_div_fp_ul (k1, 2)")
        lines.append("    let (local k2_state : Dynamics)      = dynamics_add(state, k1_half)")
        lines.append("    let (local k2_state_diff : Dynamics) = eval (k2_state)")
        lines.append("    let (local k2 : Dynamics)            = dynamics_mul_fp (k2_state_diff, dt)")
        lines.append("")
        lines.append("    # k3 stage")
        lines.append("    let (local k2_half : Dynamics)       = dynamics_div_fp_ul (k2, 2)")
        lines.append("    let (local k3_state : Dynamics)      = dynamics_add(state, k2_half)")
        lines.append("    let (local k3_state_diff : Dynamics) = eval (k3_state)")
        lines.append("    let (local k3 : Dynamics)            = dynamics_mul_fp (k3_state_diff, dt)")
        lines.append("")
        lines.append("    # k4 stage")
        lines.append("    let (local k4_state : Dynamics)      = dynamics_add(state, k3)")
        lines.append("    let (local k4_state_diff : Dynamics) = eval (k4_state)")
        lines.append("    let (local k4 : Dynamics)            = dynamics_mul_fp (k4_state_diff, dt)")
        lines.append("")
        lines.append("    # sum k, mul dt, div 6, obtain state_nxt")
        lines.append("    let (local k2_2)        = dynamics_mul_fp_ul (k2, 2)")
        lines.append("    let (local k3_2)        = dynamics_mul_fp_ul (k3, 2)")
        lines.append("    let (local sum_k1_2k2)  = dynamics_add (k1, k2_2) # wish we could overload operators..")
        lines.append("    let (local sum_2k3_k4)  = dynamics_add (k3_2, k4)")
        lines.append("    let (local k_sum)       = dynamics_add (sum_k1_2k2, sum_2k3_k4)")
        lines.append("    let (local state_delta) = dynamics_div_fp_ul (k_sum, 6)")
        lines.append("    let (local state_nxt)   = dynamics_add (state, state_delta)")
        lines.append("")
        lines.append("    return (state_nxt)")
        lines.append("end")
        lines.append("")
        
        return lines
        
    def _gen_eval(self):
        lines = []
        
        lines.append("# Problem-specific evaluation function for first-order derivative of state")
        lines.append("# TODO: Implement this to your desire")
        lines.append("func eval {range_check_ptr} (")
        lines.append("        state : Dynamics")
        lines.append("    ) -> (")
        lines.append("        state_diff : Dynamics")
        lines.append("    ):")
        lines.append("    alloc_locals")
        lines.append("    # TODO: Unpack state struct")
        for i in range(self.dim):
            lines.append(f"    local #TODO = state.q{i+1}")
            lines.append(f"    local #TODO = state.q{i+1}d")
        lines.append("    # TODO: Scene setup")
        lines.append("    # TODO: Externalize these into storage vars once block time is more mangeable")
        lines.append("    # TODO: Compute evaluation - first-order derivative of state")
        lines.append("    local state_diff : Dynamics = Dynamics(")
        for i in range(self.dim):
            lines.append(f"        q{i} = # TODO,")
            lines.append(f"        q{i+1}d = # TODO,")
        lines[-1] = lines[-1][0:-1] # remove last comma
        lines.append("    )")
        lines.append("    return (state_diff)")
        lines.append("end")
        lines.append("")
        
        return lines
    
    def _gen_fp_util(self):
        lines = []
        
        lines.append("### Utility functions for fixed-point arithmetic")
        lines.append("")
        lines.append("func mul_fp {range_check_ptr} (")
        lines.append("        a : felt,")
        lines.append("        b : felt")
        lines.append("    ) -> (")
        lines.append("        c : felt")
        lines.append("    ):")
        lines.append("    # signed_div_rem by SCALE_FP after multiplication")
        lines.append("    tempvar product = a * b")
        lines.append("    let (c, _) = signed_div_rem(product, SCALE_FP, RANGE_CHECK_BOUND)")
        lines.append("    return (c)")
        lines.append("end")
        lines.append("")
        lines.append("func div_fp {range_check_ptr} (")
        lines.append("        a : felt,")
        lines.append("        b : felt")
        lines.append("    ) -> (")
        lines.append("        c : felt")
        lines.append("    ):")
        lines.append("    # multiply by SCALE_FP before signed_div_rem")
        lines.append("    tempvar a_scaled = a * SCALE_FP")
        lines.append("    let (c, _) = signed_div_rem(a_scaled, b, RANGE_CHECK_BOUND)")
        lines.append("    return (c)")
        lines.append("end")
        lines.append("")
        lines.append("func mul_fp_ul {range_check_ptr} (")
        lines.append("        a : felt,")
        lines.append("        b_ul : felt")
        lines.append("    ) -> (")
        lines.append("        c : felt")
        lines.append("    ):")
        lines.append("    let c = a * b_ul")
        lines.append("    return (c)")
        lines.append("end")
        lines.append("")
        lines.append("func div_fp_ul {range_check_ptr} (")
        lines.append("        a : felt,")
        lines.append("        b_ul : felt")
        lines.append("    ) -> (")
        lines.append("        c : felt")
        lines.append("    ):")
        lines.append("    let (c, _) = signed_div_rem(a, b_ul, RANGE_CHECK_BOUND)")
        lines.append("    return (c)")
        lines.append("")
        
        return lines
    
    def _gen_dynamics_util(self):
        lines = []
        
        # Dynamics struct declaration
        lines.append("# Generated problem-specific struct for holding the coordinates for dynamics (all in fixed-point representation)")
        lines.append("struct Dynamics:")
        for i in range(self.dim):
            lines.append(f"    member q{i+1}  : felt")
            lines.append(f"    member q{i+1}d : felt")
        lines.append("end")
        lines.append("")
        
        # Dynamics add
        lines.append("# Generated function to compute the sum of two Dynamics structs")
        lines.append("func dynamics_add {range_check_ptr} (")
        lines.append("        state_a : Dynamics,")
        lines.append("        state_b : Dynamics")
        lines.append("    ) -> (")
        lines.append("        state_z : Dynamics")
        lines.append("    ):")
        lines.append("    alloc_locals")
        temp_list = []
        for i in range(self.dim):
            lines.append(f"    local q{i+1}_  = state_a.q{i+1}  + state_b.q{i+1}")
            lines.append(f"    local q{i+1}d_ = state_a.q{i+1}d + state_b.q{i+1}d")
            temp_list.append(f'q{i+1}=q{i+1}_')
            temp_list.append(f'q{i+1}d=q{i+1}d_')
        temp_str = ", ".join(temp_list)
        lines.append(f"    local state_z : Dynamics = Dynamics({temp_str})")
        lines.append("    return (state_z)")
        lines.append("end")
        lines.append("")
        
        # Dynamics * fp
        lines.append("# Generated function to compute a Dynamics struct multiplied by a fixed-point value")
        lines.append("func dynamics_mul_fp {range_check_ptr} (")
        lines.append("        state_a : Dynamics,")
        lines.append("        multiplier_fp  : felt")
        lines.append("    ) -> (")
        lines.append("        state_z : Dynamics")
        lines.append("    ):")
        lines.append("    alloc_locals")
        for i in range(self.dim):
            lines.append(f"    local q{i+1}  = state_a.q{i+1}")
            lines.append(f"    local q{i+1}d = state_a.q{i+1}d")
        for i in range(self.dim):
            lines.append(f"    let (local q{i+1}_)  = mul_fp (q{i+1},  multiplier_fp)")
            lines.append(f"    let (local q{i+1}d_) = mul_fp (q{i+1}d, multiplier_fp)")
            temp_list.append(f'q{i+1}=q{i+1}_')
            temp_list.append(f'q{i+1}d=q{i+1}d_')
        temp_str = ", ".join(temp_list)
        lines.append(f"    local state_z : Dynamics = Dynamics({temp_str})")
        lines.append("    return (state_z)")
        lines.append("end")
        lines.append("")
        
        # Dynamics * ul
        lines.append("# Generated function to compute a Dynamics struct multiplied by a unit-less value")
        lines.append("func dynamics_mul_fp_ul {range_check_ptr} (")
        lines.append("        state_a : Dynamics,")
        lines.append("        multiplier_ul  : felt")
        lines.append("    ) -> (")
        lines.append("        state_z : Dynamics")
        lines.append("    ):")
        lines.append("    alloc_locals")
        for i in range(self.dim):
            lines.append(f"    local q{i+1}  = state_a.q{i+1}")
            lines.append(f"    local q{i+1}d = state_a.q{i+1}d")
        for i in range(self.dim):
            lines.append(f"    let (local q{i+1}_)  = mul_fp_ul (q{i+1},  multiplier_ul)")
            lines.append(f"    let (local q{i+1}d_) = mul_fp_ul (q{i+1}d, multiplier_ul)")
            temp_list.append(f'q{i+1}=q{i+1}_')
            temp_list.append(f'q{i+1}d=q{i+1}d_')
        temp_str = ", ".join(temp_list)
        lines.append(f"    local state_z : Dynamics = Dynamics({temp_str})")
        lines.append("    return (state_z)")
        lines.append("end")
        lines.append("")
        
        # Dynamics / ul
        lines.append("# Generated function to compute a Dynamics struct divided by a unit-less value")
        lines.append("func dynamics_div_fp_ul {range_check_ptr} (")
        lines.append("        state_a : Dynamics,")
        lines.append("        divisor_ul  : felt")
        lines.append("    ) -> (")
        lines.append("        state_z : Dynamics")
        lines.append("    ):")
        lines.append("    alloc_locals")
        for i in range(self.dim):
            lines.append(f"    local q{i+1}  = state_a.q{i+1}")
            lines.append(f"    local q{i+1}d = state_a.q{i+1}d")
        for i in range(self.dim):
            lines.append(f"    let (local q{i+1}_)  = div_fp_ul (q{i+1},  divisor_ul)")
            lines.append(f"    let (local q{i+1}d_) = div_fp_ul (q{i+1}d, divisor_ul)")
            temp_list.append(f'q{i+1}=q{i+1}_')
            temp_list.append(f'q{i+1}d=q{i+1}d_')
        temp_str = ", ".join(temp_list)
        lines.append(f"    local state_z : Dynamics = Dynamics({temp_str})")
        lines.append("    return (state_z)")
        lines.append("end")
        lines.append("")
        
        return lines

### Usage
Take `dim=2` as example -- can be two-body 1D or one-body 2D.

In [33]:
rk4 = RK4CairoBuilder(dim=2)
rk4.generate()

======== GENERATED CAIRO PROGRAM BELOW ========
# Problem-specific evaluation function for first-order derivative of state
# TODO: Implement this to your desire
func eval {range_check_ptr} (
        state : Dynamics
    ) -> (
        state_diff : Dynamics
    ):
    alloc_locals
    # TODO: Unpack state struct
    local #TODO = state.q1
    local #TODO = state.q1d
    local #TODO = state.q2
    local #TODO = state.q2d
    # TODO: Scene setup
    # TODO: Externalize these into storage vars once block time is more mangeable
    # TODO: Compute evaluation - first-order derivative of state
    local state_diff : Dynamics = Dynamics(
        q0 = # TODO,
        q1d = # TODO,
        q1 = # TODO,
        q2d = # TODO
    )
    return (state_diff)
end

# Generated Runge-Kutta 4th-order method for Dynamics state
func rk4 {range_check_ptr} (
        t : felt,
        dt : felt,
        state : Dynamics
    ) -> (
        state_nxt : Dynamics
    ):
    alloc_locals
    # k1 stage
    local k1